In [2]:
vertex_names = ['I', 'L', 'O', 'V', 'E', 'D', 'N','A']
V = len(vertex_names)
edges = [(0,1),(0,2),(0,5),(1,2),
         (2,3),(1,7),(3,2),(3,5),
         (3,4),(4,5),(5,1),(5,6),(6,7)]

pairings = {'A':'T',
           'T':'A',
           'G':'C',
           'C':'G'}

K = 10

In [3]:
import random

In [10]:
vertex_strands = [''.join([random.choice(list(pairings.keys()))
                          for _ in range(K)])for _ in range(V)]
print(f"Vertex Strands Generated:{vertex_strands}")

Vertex Strands Generated:['AGAGCTAACG', 'GAAAGGGCGC', 'CTCACCCCGC', 'AGTTGGAAAC', 'GGGAATATGC', 'GAACCCCTGA', 'CAGCACTGAC', 'AGGAGAGGCG']


In [16]:
def find_edge_strands(vertices, edges):
    edge_strands = [vertices[v1][(K//2):] + 
                    vertices[v2][:(K//2)]for (v1, v2) in edges]
    return [strand.replace(vertices[0][-(K//2):], 
                           vertices[0]).replace(vertices[-1][:(K//2)],
                                                vertices[-1])
            for strand in edge_strands]
edge_strands = find_edge_strands(vertex_strands, edges)
print(f"\nEdge Strands Created: {edge_strands}")


Edge Strands Created: ['AGAGCTAACGGAAAG', 'AGAGCTAACGCTCAC', 'AGAGCTAACGGAACC', 'GGCGCCTCAC', 'CCCGCAGTTG', 'GGCGCAGGAGAGGCG', 'GAAACCTCAC', 'GAAACGAACC', 'GAAACGGGAA', 'TATGCGAACC', 'CCTGAGAAAG', 'CCTGACAGCA', 'CTGACAGGAGAGGCG']


In [ ]:
# 后5替换前5 后5替换前5 后5替换前5 后5替换前5 后5扩增前5  后5扩增前5

#靠近前1或者后1路径，就只能按照前5 或后5加完整路径

In [17]:
def find_complement(strand): # 转制为补链
    output = ""
    for nuc in strand: output += pairings[nuc]
    return output

complements = [find_complement(v)for v in vertex_strands]
print(f"\nComplements Created: {complements}")


Complements Created: ['TCTCGATTGC', 'CTTTCCCGCG', 'GAGTGGGGCG', 'TCAACCTTTG', 'CCCTTATACG', 'CTTGGGGACT', 'GTCGTGACTG', 'TCCTCTCCGC']


In [19]:
all_edges = edge_strands * 10000 #扩充样本
random.shuffle(all_edges)
  #打散样本
path_strands = []
growing_strand = ""

for idx, s in enumerate(all_edges):#枚举 idx为索引号 s为碱基
    if len(growing_strand) == 0:
        growing_strand +=s
    else:
        target_comp = find_complement(growing_strand[-(K//2):]) + find_complement(s[:(K//2)])
        #找现在成长路径后5个碱基与s的前五个相加组成target碱基
        if target_comp in complements: # ？complements？
            growing_strand += s
    
    if growing_strand[-K:] == vertex_strands[-1]: #10位之后的几位等于标准序列倒数第一
        path_strands.append(growing_strand) #增加进路径
        growing_strand = "" #设置为空

print(f"Strands Created: {len(path_strands)}")

Strands Created: 3466


### step2 : Remove paths without correct start & end

In [20]:
in_and_out_strands = [path for path in path_strands if path[:K] == vertex_strands[0] and path[-K:] == vertex_strands[-1]]
print(f"\n Strands Strarting at {vertex_names[0]} and Ending at {vertex_names[V-1]}: {len(in_and_out_strands)}")

#头尾和序列头尾不一样的，删除掉


 Strands Strarting at I and Ending at A: 816


### step 3: Keep paths with N vertices

In [23]:
n_step_paths = [path for path in in_and_out_strands if len(path) == V*10]
print(f"\nStrands with {V} Steps:{len(n_step_paths)}")

#选出长度符合八个数据的碱基变体 8*10 碱基为10


Strands with 8 Steps:82


### step4: Keep onlu path that touch each vertex once

In [25]:
included = n_step_paths
for i in range(len(vertex_strands)):
    included = [path for path in included if vertex_strands[i] in path]
    if i == 0 or i == len(vertex_strands) - 1:
        print(f"- Already Checked for {vertex_names[i]}")
    else:
        print(f"- Eliminating Paths Not Including {vertex_names[i]}: {len(included)} Remaining")
print(f"Strands Including All Vertices >= Once: {len(included)}")
# 检查是否重复 或者是否有序

- Already Checked for I
- Eliminating Paths Not Including L: 65 Remaining
- Eliminating Paths Not Including O: 65 Remaining
- Eliminating Paths Not Including V: 65 Remaining
- Eliminating Paths Not Including E: 41 Remaining
- Eliminating Paths Not Including D: 41 Remaining
- Eliminating Paths Not Including N: 31 Remaining
- Already Checked for A
Strands Including All Vertices >= Once: 31


### step5: extract any path that remain

In [32]:
solution = included[0]
# 将筛选出来的路径转码
def decode_solution(strand):
    path = []
    for i in range(0, len(strand), 10):
        s = strand[i: i+10]
        vertex_num = vertex_strands.index(s)
        decoded_letter = vertex_names[vertex_num]
        path.append(decoded_letter)
    return path

path = decode_solution(solution)
print(f"\n Solution: {str(path)}")


 Solution: ['I', 'L', 'O', 'V', 'E', 'D', 'N', 'A']


In [ ]:
"""
思路 用坐标系的方式 给零件用碱基进行编码，
位置信息也可以用ATCG编码，分别为上下左右，
设置规矩，然后随机生成
筛选掉超出规模的，筛选掉零件超过限制的，筛选掉剩余面积过多的，
筛选掉每个工件都用到的，筛选掉总板料用的多的。
随着随机生成量越多，得到全局最优越强
"""